In [15]:
import numpy as np
import re
import os
import pandas as pd
from termcolor import colored
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
import json
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import glob
import os.path
import numpy as np
import sys
import codecs
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.externals import joblib

In [16]:
def clean(text):
    text=text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text=re.sub('[“"”]',' " ',text)
    retain='[^abcdefghijklmnopqrstuvwxyz!#?" ]'
    text=re.sub('[()–-]',' ',text)
    text=re.sub(retain,'',text)
    text=text.replace('?',' ? ')
    text=text.replace('#',' # ')
    text=text.replace('!',' ! ')
    text=text.split()
    return ' '.join(text)

In [4]:
directory='datasets/train-task2-TC.labels'
props_=open(directory).read().split('\n')[:-1]
sentences,Articles, labels=[],[],[]
IDS=[]
Spans=[]
for prop in props_:
    prop_=prop.split('\t')
    x,y=int(prop_[2]),int(prop_[3])
    Spans.append((x,y))
    sentences.append([open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[:x],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[x:y],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[y:]])
    Articles.append(open('datasets/train-articles/article{}.txt'.format(prop_[0])).read())
    labels.append(prop_[1])
    IDS.append(prop_[0])
    ids=int(prop_[0])

In [5]:
ind=2
def gen(ind):
    print(labels[ind])
    pre=colored(sentences[ind][0], color='green')
    sen=colored(sentences[ind][1], color='red')
    post=colored(sentences[ind][-1], color='green')
    print(pre+sen+post)
def span_read(id,span):
    return open('datasets/train-articles/article{}.txt'.format(id)).read()[span[0]:span[1]]
def freq_dict(text,Stem=False):
    freq={}
    for word in clean(text,Stem).split():
        if word in freq:
            freq[word]=freq[word]+1
        else:
            freq[word]=1
    return freq

In [6]:
df=pd.DataFrame({'Id':IDS, 'Span':Spans, 'Label':labels})
set(labels)

{'Appeal_to_Authority',
 'Appeal_to_fear-prejudice',
 'Bandwagon,Reductio_ad_hitlerum',
 'Black-and-White_Fallacy',
 'Causal_Oversimplification',
 'Doubt',
 'Exaggeration,Minimisation',
 'Flag-Waving',
 'Loaded_Language',
 'Name_Calling,Labeling',
 'Repetition',
 'Slogans',
 'Thought-terminating_Cliches',
 'Whataboutism,Straw_Men,Red_Herring'}

In [7]:
dumps=True
if(not dumps):
    Stitch=""
    for doc in os.listdir('datasets/train-articles'):
        Stitch=Stitch+open('datasets/train-articles/'+doc).read()+' '    
    f = open("dumps/Text_stitch.txt","w")
    f.write(Stitch)
    f.close()
    corp_freq=dict()
    Doc_freqs=dict()
    corp_freq=freq_dict(Stitch)
    for id in IDS :
        txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
        Doc_freqs[id]=freq_dict(txt)

    corp_freq_json = json.dumps(corp_freq)
    f = open("dumps/Corp_freq_raw.json","w")
    f.write(corp_freq_json)
    f.close()

    Doc_freqs_json = json.dumps(Doc_freqs)
    f = open("dumps/Doc_freqs_raw.json","w")
    f.write(Doc_freqs_json)
    f.close()

    corp_freq_stem=dict()
    Doc_freqs_stem=dict()
    corp_freq_stem=freq_dict(Stitch,Stem=True)
    for id in IDS :
        txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
        Doc_freqs_stem[id]=freq_dict(txt,Stem=True)

    corp_freq_json_stem = json.dumps(corp_freq_stem)
    f = open("dumps/Corp_freq_stem.json","w")
    f.write(corp_freq_json_stem)
    f.close()

    Doc_freqs_json_stem = json.dumps(Doc_freqs_stem)
    f = open("dumps/Doc_freqs_stem.json","w")
    f.write(Doc_freqs_json_stem)
    f.close()

try: 
    Stitch 
except: 
    Stitch=open("dumps/Text_stitch.txt",'r').read()
try: 
    corp_freq_raw 
except: 
    corp_freq_raw=json.loads(open("dumps/Corp_freq_raw.json",'r').read())
try: 
    Doc_freqs_raw 
except: 
    Doc_freqs_raw=json.loads(open("dumps/Doc_freqs_raw.json",'r').read())
try: 
    corp_freq_stem 
except: 
    corp_freq_stem=json.loads(open("dumps/Corp_freq_stem.json",'r').read())
try: 
    Doc_freqs_stem 
except: 
    Doc_freqs_stem=json.loads(open("dumps/Doc_freqs_stem.json",'r').read())

In [8]:
df

,Id,Span,Label
0,111111111,"(265, 323)",Appeal_to_Authority
1,111111111,"(1795, 1935)",Appeal_to_Authority
2,111111111,"(149, 157)",Doubt
3,111111111,"(1069, 1091)",Repetition
4,111111111,"(1334, 1462)",Appeal_to_fear-prejudice
5,111111111,"(1577, 1616)",Appeal_to_fear-prejudice
6,111111111,"(1856, 1910)",Appeal_to_fear-prejudice
7,111111111,"(2023, 2086)",Appeal_to_fear-prejudice
8,111111112,"(191, 219)",Slogans
9,111111112,"(476, 556)",Black-and-White_Fallacy


In [17]:
train_folder = "datasets/train-articles" # check that the path to the datasets folder is correct, 
dev_folder = "datasets/dev-articles"     # if not adjust these variables accordingly
train_labels_file = "datasets/train-task2-TC.labels"
dev_template_labels_file = "datasets/dev-task-TC-template.out"
task_TC_output_file = "baseline-output-TC.txt"

def read_articles_from_file_list(folder_name, file_pattern="*.txt"):
    """
    Read articles from files matching patterns <file_pattern> from  
    the directory <folder_name>. 
    The content of the article is saved in the dictionary whose key
    is the id of the article (extracted from the file name).
    Each element of <sentence_list> is one line of the article.
    """
    file_list = glob.glob(os.path.join(folder_name, file_pattern))
    articles = {}
    article_id_list, sentence_id_list, sentence_list = ([], [], [])
    for filename in sorted(file_list):
        article_id = os.path.basename(filename).split(".")[0][7:]
        with codecs.open(filename, "r", encoding="utf8") as f:
            articles[article_id] = f.read()
    return articles


def read_predictions_from_file(filename):
    """
    Reader for the gold file and the template output file. 
    Return values are four arrays with article ids, labels 
    (or ? in the case of a template file), begin of a fragment, 
    end of a fragment. 
    """
    articles_id, span_starts, span_ends, gold_labels = ([], [], [], [])
    with open(filename, "r") as f:
        for row in f.readlines():
            article_id, gold_label, span_start, span_end = row.rstrip().split("\t")
            articles_id.append(article_id)
            gold_labels.append(gold_label)
            span_starts.append(span_start)
            span_ends.append(span_end)
    return articles_id, span_starts, span_ends, gold_labels

def compute_features_b(articles, span_starts, span_ends):
    # only one feature, the length of the span
    return np.array([ int(sp_ends)-int(sp_starts) for sp_starts, sp_ends in zip(span_starts, span_ends) ]).reshape(-1, 1)

def clean(text):
    text=text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text=re.sub('[“"”]',' " ',text)
    retain='[^abcdefghijklmnopqrstuvwxyz!#?". ]'
    text=re.sub('[()–-]',' ',text)
    text=re.sub(retain,'',text)
    text=re.sub('[.]',' . ',text)
    text=text.replace('?',' ? ')
    text=text.replace('#',' # ')
    text=text.replace('!',' ! ')
    return ' '.join(text.split())

# loading articles' content from *.txt files in the train folder
articles = read_articles_from_file_list(train_folder)
dev_articles = read_articles_from_file_list(dev_folder)

def read_span(id,span,dev=False):
    if dev:
        return dev_articles[id][span[0]:span[1]]
    else:
        return articles[id][span[0]:span[1]]

In [18]:
in_data=[(id, [int(sps),int(spe)])for id, sps, spe in zip(ref_articles_id, ref_span_starts, ref_span_ends)]
df=pd.DataFrame(in_data,columns=['ID','Span'])
df['Sentence']=[read_span(id,span) for id,span in zip(df['ID'].tolist(),df['Span'].tolist())]
df['Sentence']=df['Sentence'].apply(lambda x : clean(x))
df['Span']=df['Span'].apply(lambda x : x[1]-x[0])
df['Target']=train_gold_labels
df=df.drop_duplicates()

In [10]:
### MAIN ###

# loading articles' content from *.txt files in the train folder
articles = read_articles_from_file_list(train_folder)

In [11]:
# loading gold labels, articles ids and sentence ids from files *.task-TC.labels in the train labels folder 
ref_articles_id, ref_span_starts, ref_span_ends, train_gold_labels = read_predictions_from_file(train_labels_file)
print("Loaded %d annotations from %d articles" % (len(ref_span_starts), len(set(ref_articles_id))))

Loaded 6129 annotations from 357 articles


In [13]:
# compute one feature for each fragment, i.e. the length of the fragment, and train the model
train = compute_features(articles, ref_span_starts, ref_span_ends)

In [14]:
model = LogisticRegression(penalty='l2', class_weight='balanced', solver="lbfgs")
model.fit(train, train_gold_labels)

# reading data from the development set
dev_articles = read_articles_from_file_list(dev_folder)
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)

# computing the predictions on the development set
dev = compute_features(dev_articles, dev_span_starts, dev_span_ends)
predictions = model.predict(dev)

# writing predictions to file
with open(task_TC_output_file, "w") as fout:
    for article_id, prediction, span_start, span_end in zip(dev_article_ids, predictions, dev_span_starts, dev_span_ends):
        fout.write("%s\t%s\t%s\t%s\n" % (article_id, prediction, span_start, span_end))
print("Predictions written to file " + task_TC_output_file)

/home/somesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Predictions written to file baseline-output-TC.txt


In [20]:
lambda
df[df['Target']=='Appeal_to_Authority']

,ID,Span,Sentence,Target
0,111111111,58,the next transmission could be more pronounced...,Appeal_to_Authority
1,111111111,140,when the plague comes again it starts from mor...,Appeal_to_Authority
31,111111115,90,"frankens office said he has not "" intentionall...",Appeal_to_Authority
49,111111122,124,as the political scientist richard neustadt sa...,Appeal_to_Authority
149,111111135,412,"edward e . curtis iv the author of "" black mus...",Appeal_to_Authority
198,694811415,48,i can tell you this is not that kind of guy,Appeal_to_Authority
202,695833178,78,laura loomer proves that authorities have dire...,Appeal_to_Authority
208,695833178,117,according to loomer she obtained the image fro...,Appeal_to_Authority
263,697454736,184,pope francis explains this again in al when he...,Appeal_to_Authority
269,697454736,137,i have read the cardinals exposition in both i...,Appeal_to_Authority
